# Neptune as Graph Memory

In this notebook, we will be connecting using a AWS Neptune Analytics instance as our memory graph storage for Mem0.

The Graph Memory storage persists memories in a graph or relationship form when performing `m.add` memory operations. It then uses vector distance algorithms to find related memories during a `m.search` operation. Relationships are returned in the result, and add context to the memories.

References

## Prerequisites

### 1. Install Mem0 with Graph Memory support 

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### 2. Connect to Neptune

To connect to AWS Neptune Analytics, you need to configure Neptune with your AWS profile credentials. The best way to do this is to declare environment variables with IAM permission to your Neptune Analytics instance. The `graph-identifier` for the instance to persist memories needs to be defined in the Mem0 configuration under `"graph_store"`, with the `"neptune"` provider.  Note that the Neptune Analytics instance needs to have `vector-search-configuration` defined to meet the needs of the llm model's vector dimensions.

```python
embedding_dimensions = 1536
graph_identifier = "<MY-GRAPH>" # graph with 1536 dimensions for vector search
config = {
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-large",
            "embedding_dims": embedding_dimensions
        },
    },
    "graph_store": {
        "provider": "neptune",
        "config": {
            "graph_identifier": graph_identifier,
        },
    },
}
```

### 3. Configure OpenSearch

We're going to use OpenSearch as our vector store.  You can run [OpenSearch from docker image](https://docs.opensearch.org/docs/latest/install-and-configure/install-opensearch/docker/):

```bash
docker pull opensearchproject/opensearch:3
```

And verify that it's running with a `<custom-admin-password>`:

```bash
 docker run -d -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" -e "OPENSEARCH_INITIAL_ADMIN_PASSWORD=<custom-admin-password>" opensearchproject/opensearch:latest

 curl https://localhost:9200 -ku admin:<custom-admin-password>
```

We're going to connect [OpenSearch using the python client](https://github.com/opensearch-project/opensearch-py):

```bash
pip install "opensearch-py"
```

## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [ ]:
from mem0 import Memory
from opensearchpy import OpenSearch
from langchain_aws import NeptuneAnalyticsGraph
import os, logging, sys

logging.getLogger('mem0.memory.neptune_memory').setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logging.basicConfig(
    format='%(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout  # Explicitly set output to stdout
)

Setup the Mem0 configuration using:
- openai as the embedder
- AWS Neptune Analytics instance as a graph store
- OpenSearch as the vector store

In [ ]:
graph_identifier = os.environ.get("GRAPH_ID")
opensearch_username = os.environ.get("OS_USERNAME")
opensearch_password = os.environ.get("OS_PASSWORD")
config = {
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-large",
            "embedding_dims": 1536
        },
    },
    "graph_store": {
        "provider": "neptune",
        "config": {
            "graph_identifier": graph_identifier,
        },
    },
    "vector_store": {
        "provider": "opensearch",
        "config": {
            "collection_name": "vector_store",
            "host": "localhost",
            "port": 9200,
            "user": opensearch_username,
            "password": opensearch_password,
            "use_ssl": False,
            "verify_certs": False,
        },
    },
}

## Graph Memory initializiation

Initialize Memgraph as a Graph Memory store:

In [ ]:
m = Memory.from_config(config_dict=config)

app_id = "movies"
user_id = "alice"

## Store memories

Create memories and store:

In [ ]:
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]

# Store inferred memories (default behavior)
result = m.add(messages, user_id=user_id, metadata={"category": "movie_recommendations"})


## Search memories

In [ ]:
search_results = m.search("what does alice love?", user_id=user_id)
for result in search_results["results"]:
    print(f"\"{result["memory"]}\" [score: {result["score"]}]")
for relation in search_results["relations"]:
    print(f"{relation}")

Neptune Analytics Graph

In [ ]:
all_results = m.get_all(user_id=user_id)
for n in all_results['results']:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results['relations']:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

In [ ]:
neptune_graph = NeptuneAnalyticsGraph(graph_identifier)

query = """
        MATCH (n {user_id: $user_id})-[r]->(m {user_id: $user_id})
        RETURN n.name AS source, type(r) AS relationship, m.name AS target
        LIMIT $limit
        """
edge_results = neptune_graph.query(
    query, params={"user_id": user_id, "limit": 100}
)

print("----RELATIONSHIPS----")
for e in edge_results:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

query = """
MATCH (n {user_id: $user_id})
CALL neptune.algo.vectors.get(n)
YIELD embedding
RETURN n AS node, embedding
LIMIT $limit
"""

node_results = neptune_graph.query(
    query, params={"user_id": user_id, "limit": 100}
)
# print(f"node_results={node_results}")

print("----NODES----")
for n in node_results:
    has_embedding = n.get("embedding", None) is not None
    print(f"node name:\"{n["node"]["~properties"]["name"]}\" embedding: {has_embedding}")



In [ ]:
opensearch_index = "vector_store"
vector_store_config = config[opensearch_index]["config"]

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': vector_store_config["host"], 'port': vector_store_config["port"]}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (vector_store_config["user"], vector_store_config["password"]),
    use_ssl = vector_store_config["use_ssl"],
    verify_certs = vector_store_config["verify_certs"],
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

query = {"query":{"match_all": {}}}

response = client.search(
    body = query,
    index = opensearch_index,
)

print("----VECTORS----")
for v in response["hits"]["hits"]:
    print(f"vector id=\"{v["_source"]["id"]}\" data={v["_source"]["payload"]["data"]}")

In [ ]:
m.reset()
# only works for neptune_memory
memory_graph = m.graph.reset()